In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import stats
import statsmodels.formula.api as smf
from IPython.display import display

%precision 3

'%.3f'

In [5]:
df = pd.read_csv('../data/ch12_scores_reg.csv')
n = len(df)
print(f'標本の大きさ: {n}')
df.head()

標本の大きさ: 20


,小テスト,期末テスト,睡眠時間,通学方法
0,4.2,67,7.2,バス
1,7.2,71,7.9,自転車
2,0.0,19,5.3,バス
3,3.0,35,6.8,徒歩
4,1.5,35,7.5,徒歩
